In [1]:
import datetime
import os
import seaborn as sns
import sys
import time

%matplotlib inline

In [2]:
SCRIPTS_DIR = '../app'

sys.path.insert(1, SCRIPTS_DIR)

## Objective: New features to MA Trader 
### [1] Feat: MaxDrawdown as metric and NumExecution for information display [100%]
### [2] Feat: new trading strategy based upon double moving averages [100%]
### [3] Feat: new trading strategy w/ EMA [10%]
### [4] Feat: new trading strategy w/ MACD

#### Prepare Data

In [3]:
# config

SECONDS_IN_ONE_DAY = 86400

STRATEGIES = ['MA-SELVES', 'DOUBLE-MA']
#STRATEGIES = ['DOUBLE-MA']

# simulation configuration
TOL_PCTS = [0.08, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
BUY_PCTS = [0.3, 0.4, 0.5, 0.6, 0.7]
SELL_PCTS = [0.3, 0.4, 0.5, 0.6, 0.7]

MA_LENGTHS = [3, 5, 10, 20, 30]
EMA_LENGTHS = [10,26]

BUY_STAS = ('by_percentage')
SELL_STAS = ('by_percentage')

# currencies (digital + real-world)
CURS = ['BTC', 'ETH', 'LTC', 'BCH', 'ETC']
FIAT = ['USD']

# signals
NO_ACTION_SIGNAL = 'NO ACTION'
BUY_SIGNAL = 'BUY'
SELL_SIGNAL = 'SELL'

# epilson
EP_COIN = 10e-3
EP_CASH = 5

In [4]:
from trader_driver import TraderDriver
from util import load_csv

name,l = load_csv(csv_fn='../tests/fixtures/BTC_HISTORY.csv')

l = l[-100:]

In [5]:
init_amount = 3000
cur_coin = 5
mode = 'normal'

t_driver = TraderDriver(
    name=name,
    init_amount=init_amount,
    overall_stats=STRATEGIES,
    cur_coin=cur_coin,
    tol_pcts=TOL_PCTS,
    ma_lengths=MA_LENGTHS,
    ema_lengths=EMA_LENGTHS,
    buy_pcts=BUY_PCTS,
    sell_pcts=SELL_PCTS,
    buy_stas=BUY_STAS,
    sell_stas=SELL_STAS,
    mode=mode
)

t_driver.feed_data(data_stream=l)

info = t_driver.best_trader_info
# best trader
best_t = t_driver.traders[info['trader_index']]
# for a new price, find the trade signal
signal = best_t.trade_signal

signal

--> executed=feed_data using time=2.27 seconds.


{'action': 'NO ACTION', 'buy_percentage': 0.7, 'sell_percentage': 0.7}

In [6]:
best_t.all_history[:2]

[{'action': 'NO ACTION',
  'price': 9133.7778303426,
  'date': '2020-07-17',
  'coin': 5,
  'cash': 3000,
  'portfolio': 48668.889151713},
 {'action': 'NO ACTION',
  'price': 9160.4105914881,
  'date': '2020-07-18',
  'coin': 5,
  'cash': 3000,
  'portfolio': 48802.0529574405}]

In [7]:
best_t.all_history_trade_only

[{'action': 'SELL',
  'price': 11187.779726961699,
  'date': '2020-07-28',
  'coin': 1.5,
  'cash': 41374.084463478626,
  'portfolio': 58155.75405392118},
 {'action': 'SELL',
  'price': 11187.779726961699,
  'date': '2020-07-28',
  'coin': 0.4500000000000002,
  'cash': 52886.30980252221,
  'portfolio': 57920.810679654984},
 {'action': 'SELL',
  'price': 10939.6701415471,
  'date': '2020-07-29',
  'coin': 0.13500000000000006,
  'cash': 56263.3859752178,
  'portfolio': 57740.24144432666},
 {'action': 'SELL',
  'price': 11284.4589868467,
  'date': '2020-07-30',
  'coin': 0.04050000000000002,
  'cash': 57308.43972198968,
  'portfolio': 57765.46031095697},
 {'action': 'SELL',
  'price': 11766.7482970949,
  'date': '2020-08-02',
  'coin': 0.012150000000000008,
  'cash': 57635.355289927866,
  'portfolio': 57778.32128173757},
 {'action': 'SELL',
  'price': 11139.0983803526,
  'date': '2020-08-03',
  'coin': 0.0036450000000000024,
  'cash': 57728.19856101827,
  'portfolio': 57768.80057461465},


In [8]:
best_t.max_drawdown, best_t.num_transaction, best_t.high_strategy

(0.0714, 22, 'MA-SELVES')

In [9]:
info

{'buy_pct': 0.7,
 'sell_pct': 0.7,
 'tol_pct': 0.1,
 'buy': 'by_percentage',
 'sell': 'by_percentage',
 'init_value': 48668.889151713,
 'max_final_value': 72394.453,
 'rate_of_return': '48.749%',
 'baseline_rate_of_return': '40.777%',
 'coin_rate_of_return': '43.456%',
 'trader_index': 49}

In [10]:
best_t.crypto_prices[-10:]

[(11666.211700321399, '2020-10-13'),
 (11443.7327918222, '2020-10-14'),
 (11395.54736637, '2020-10-15'),
 (11504.5490067109, '2020-10-16'),
 (11355.160439450901, '2020-10-17'),
 (11358.706379629699, '2020-10-18'),
 (11471.002547787, '2020-10-19'),
 (11776.592997329499, '2020-10-20'),
 (11936.3629133377, '2020-10-21'),
 (13102.9344615485, '2020-10-22')]

In [11]:
best_t.moving_averages

{'3': [None,
  None,
  9201.758414473932,
  9168.404951575134,
  9156.958270643701,
  9175.789505261566,
  9179.8487522895,
  9252.641437550334,
  9366.147887934067,
  9514.411064202968,
  9572.381157562466,
  9631.767242478832,
  9738.958362972933,
  10278.560029973034,
  10688.8003637899,
  11137.302951785166,
  11114.349180275933,
  11258.898218640832,
  11419.6613220569,
  11426.387978029767,
  11389.217669729233,
  11209.6436787031,
  11381.080080439533,
  11559.417091522731,
  11696.720120226068,
  11727.221616872868,
  11685.990241055135,
  11750.087540389397,
  11615.3109621032,
  11574.1186543812,
  11547.432437080133,
  11694.999460618868,
  11810.359672458799,
  11858.619177112132,
  12063.895334912799,
  12122.599321311,
  12073.560100835099,
  11885.155255513333,
  11722.2886000297,
  11697.220215078632,
  11641.391139591966,
  11695.015724263765,
  11596.3829495644,
  11527.187107647767,
  11379.889863597533,
  11414.389749163767,
  11433.130360566,
  11551.424683809333,


In [12]:
best_t.exp_moving_averages

{'10': deque([]), '26': deque([])}